In [2]:
import gpxpy
import gpxpy.gpx
from tcx2gpx.tcx2gpx import TCX2GPX
import os
import xml.etree.ElementTree as ET
import gpxo
import tcxparser
from lxml import etree
import datetime 

import warnings
warnings.simplefilter(action='ignore')

In [5]:
# Formatação dos arquivos tcx com xml organizado
# Antes disso tive que tirar a identação de cada arquivo tcx nas primeiras linhas 

def format_xml_file(file_path):
    parser = etree.XMLParser(remove_blank_text=True)
    tree = etree.parse(file_path, parser)
    root = tree.getroot()
    
    # Pretty-print the XML
    pretty_xml_as_string = etree.tostring(root, pretty_print=True, encoding='unicode')
    
    # Write the formatted XML back to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(pretty_xml_as_string)

def format_xml_files_in_directory(directory_path):
    for filename in os.listdir(directory_path):
        #if filename.endswith('.xml'):
            file_path = os.path.join(directory_path, filename)
            format_xml_file(file_path)
            print(f"Formatted: {file_path}")

# Path to the directory containing XML files
directory_path = 'C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/athlete7_tcx/tcx'

# Format all XML files in the directory
format_xml_files_in_directory(directory_path)

Formatted: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/athlete7_tcx/tcx\11776505371.tcx
Formatted: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/athlete7_tcx/tcx\11801519113.tcx
Formatted: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/athlete7_tcx/tcx\11813003840.tcx
Formatted: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/athlete7_tcx/tcx\11813004309.tcx
Formatted: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/athlete7_tcx/tcx\11816096142.tcx
Formatted: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/athlete7_tcx/tcx\11827050519.tcx
Formatted: C:/Users/USER/Desktop/EstudosDados/

In [9]:
'''Código para limpar o arquivo tcx e transformar em gpx'''

def clean_tcx_file(tcx_file_path):
    """Remove caracteres extras antes da declaração XML."""
    with open(tcx_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remover linhas em branco ou caracteres extras antes da declaração XML
    for i, line in enumerate(lines):
        if line.strip().startswith('<?xml'):
            return ''.join(lines[i:])

    return ''.join(lines)

def convert_tcx_to_gpx(tcx_file_path, gpx_file_path):
    # Limpar o arquivo TCX
    clean_tcx_content = clean_tcx_file(tcx_file_path)

    # Parse the cleaned TCX content
    root = ET.fromstring(clean_tcx_content)

    # Define namespaces
    namespaces = {
        'tcx': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2'
    }

    # Create a new GPX object
    gpx = gpxpy.gpx.GPX()

    # Iterate through all Trackpoints in the TCX file
    for trackpoint in root.findall('.//tcx:Trackpoint', namespaces):
        time = trackpoint.find('tcx:Time', namespaces).text
        position = trackpoint.find('tcx:Position', namespaces)
        latitude = position.find('tcx:LatitudeDegrees', namespaces).text if position else None
        longitude = position.find('tcx:LongitudeDegrees', namespaces).text if position else None
        elevation = trackpoint.find('tcx:AltitudeMeters', namespaces).text if trackpoint.find('tcx:AltitudeMeters', namespaces) is not None else None

        # Parse the time using datetime with the correct format
        if time:
            time = datetime.datetime.strptime(time, '%Y-%m-%dT%H:%M:%S.%fZ')

        # Create a new GPX track point
        gpx_track_point = gpxpy.gpx.GPXTrackPoint(
            latitude=float(latitude) if latitude else None,
            longitude=float(longitude) if longitude else None,
            elevation=float(elevation) if elevation else None,
            time=time
        )

        # Add the track point to the GPX track
        if gpx.tracks == []:
            gpx_track = gpxpy.gpx.GPXTrack()
            gpx.tracks.append(gpx_track)
            gpx_segment = gpxpy.gpx.GPXTrackSegment()
            gpx_track.segments.append(gpx_segment)
        else:
            gpx_track = gpx.tracks[0]
            gpx_segment = gpx_track.segments[0]

        gpx_segment.points.append(gpx_track_point)

    # Write the GPX object to a file
    with open(gpx_file_path, 'w', encoding='utf-8') as gpx_file:
        gpx_file.write(gpx.to_xml())

    print(f"Arquivo GPX salvo em: {gpx_file_path}")

In [10]:
# Caminho para o arquivo TCX de entrada e o arquivo GPX de saída
tcx_folder_path = 'C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/athlete7_tcx/tcx'
gpx_folder_path = 'C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/athlete7_tcx/gpx2'

# Itera por cada arquivo na pasta
for filename in os.listdir(tcx_folder_path):
    # Junta o nome do arquivo com o caminho da pasta
    full_tcx_path = os.path.join(tcx_folder_path, filename)
    full_gpx_path = os.path.join(gpx_folder_path, filename).replace('.tcx','.gpx')    

    # Converter o arquivo TCX para GPX
    convert_tcx_to_gpx(full_tcx_path, full_gpx_path)

Arquivo GPX salvo em: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/athlete7_tcx/gpx2\11776505371.gpx
Arquivo GPX salvo em: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/athlete7_tcx/gpx2\11801519113.gpx
Arquivo GPX salvo em: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/athlete7_tcx/gpx2\11813003840.gpx
Arquivo GPX salvo em: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/athlete7_tcx/gpx2\11813004309.gpx
Arquivo GPX salvo em: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/athlete7_tcx/gpx2\11816096142.gpx
Arquivo GPX salvo em: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts/tests_and_preprocessing/athlete7

In [42]:
# Caminho da pasta que terá iteração nos seus arquivos
tcx_folder_path = 'C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/athletes_activities/athlete6/tcx'

# Contador de linhas para servir de index
count = 0

# Itera por cada arquivo na pasta
for filename in os.listdir(tcx_folder_path):
    # Junta o nome do arquivo com o caminho da pasta
    full_path = os.path.join(tcx_folder_path, filename) 
    gps_object = TCX2GPX(tcx_path="C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/athletes_activities/athlete6/tcx/10200151431.tcx")
    gps_object.convert()    
        
    # Acréscimo no contador para reiniciar o loop
    count +=1

10200151431.tcx 2023-07-24T10:03:48Z
10206600795.tcx 2023-07-25T10:01:32Z
10214037485.tcx 2023-07-26T11:02:46Z
10214334699.tcx 2023-07-26T12:14:36Z
10220741649.tcx 2023-07-27T11:24:00Z
10236087207.tcx 2023-07-29T18:50:02Z
10266648147.tcx 2023-08-03T10:02:38Z
10273274134.tcx 2023-08-04T10:01:15Z
10282905881.tcx 2023-08-05T19:08:50Z
10289101638.tcx 2023-08-06T15:44:05Z
10329620953.tcx 2023-08-12T14:55:40Z
10329620987.tcx 2023-08-12T15:40:12Z
10337340870.tcx 2023-08-13T15:28:17Z
10341533745.tcx 2023-08-14T12:08:32Z
10341533784.tcx 2023-08-14T11:16:38Z
10348272808.tcx 2023-08-15T10:11:07Z
10362592099.tcx 2023-08-17T10:03:33Z
10378681091.tcx 2023-08-19T13:22:02Z
10386953133.tcx 2023-08-20T18:28:38Z
10390209367.tcx 2023-08-21T10:05:22Z
10397154175.tcx 2023-08-22T10:02:45Z
10411633701.tcx 2023-08-24T10:01:59Z
10427746836.tcx 2023-08-26T18:05:33Z
10427746870.tcx 2023-08-26T18:31:03Z
10437961665.tcx 2023-08-28T11:05:49Z
10444074661.tcx 2023-08-29T10:07:56Z
10458087648.tcx 2023-08-31T10:03:20Z
1

In [41]:
'''
# Formatação dos arquivos tcx com xml organizado

def format_xml_file(file_path):
    parser = etree.XMLParser(remove_blank_text=True)
    tree = etree.parse(file_path, parser)
    root = tree.getroot()
    
    # Pretty-print the XML
    pretty_xml_as_string = etree.tostring(root, pretty_print=True, encoding='unicode')
    
    # Write the formatted XML back to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(pretty_xml_as_string)

def format_xml_files_in_directory(directory_path):
    for filename in os.listdir(directory_path):
        #if filename.endswith('.xml'):
            file_path = os.path.join(directory_path, filename)
            format_xml_file(file_path)
            print(f"Formatted: {file_path}")

# Path to the directory containing XML files
directory_path = 'C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/athletes_activities/athlete6/tcx'

# Format all XML files in the directory
format_xml_files_in_directory(directory_path)

'''

Formatted: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/athletes_activities/athlete6/tcx\10200151431.tcx
Formatted: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/athletes_activities/athlete6/tcx\10206600795.tcx
Formatted: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/athletes_activities/athlete6/tcx\10214037485.tcx
Formatted: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/athletes_activities/athlete6/tcx\10214334699.tcx
Formatted: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/athletes_activities/athlete6/tcx\10220741649.tcx
Formatted: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/athletes_activities/athlete6/tcx\10236087207.tcx
Formatted: C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/athletes_activities/athlete6/tcx\10266648147.tcx